In [1]:
%matplotlib ipympl
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import critical_loads as cl
import statsmodels.formula.api as sm

plt.style.use('ggplot')

In [3]:
# Connect to Docker PostGIS
eng = nivapy.da.connect(src='postgres',
                        db_name='critical_loads',
                        port = 25432)

Connection successful.


# Critical loads for vegetation (high-resolution method; 2018 onwards)

In Spring 2018, the worflow for calculating critical loads for vegetation [was refined](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow_new_grid.ipynb#1.-Vegetation) to make use of new, higher-resolution input datasets. During November 2018, data handling for the Critical Loads project was also redesigned, with the ultimate aim of centralising and migrating all key datasets onto NIVA's new cloud platform. This notebook estimates exceedences of critical loads for vegetation using both the revised (Spring 2018) workflow and the new (November 2018) data structures. Calculations here assume that deposition data are supplied using NILU's 0.1 degree grid, and the vegetation map is based on 30 m resolution **raster** satellite imagery.

**Note:** Data supplied prior to 2017-18 use a different deposition grid (the "BLR" grid) and vector vegetation data. **The workflow described here only applies to data supplied from 2017-18 onwards**; earlier data is *not* compatible with the calculations described here.    

**To do?:** Create a notebook to repeat the old workflow as well?

## 1. Upload new deposition data

The first step, if desired, is to upload new deposition data for the time period of interest.

### 1.1. Define a new deposition series

The code below extracts the existing deposition series already defined in the database. Click `Add row` to create a new row at the bottom of the table (initially a duplicate of the last row), then double-click to edit values and define the new data series. Note that all columns are mandatory.

In [ ]:
# Define a new deposition series
ser_grid = cl.view_dep_series(eng)
ser_grid

In [ ]:
# Add new series to database
add_df = cl.add_dep_series(ser_grid, eng)
add_df

### 1.2. Upload deposition data

In 2017, NILU supplied raw data for the 0.1 degree grid in `.dat` format. The code here assumes new data are supplied in the same way.

Modify the user options in the cell below to match your new data:

 * `ser_id` is the ID of the new row that was added to the table above
 
 * `dat_fold` is the *relative* path to a folder containing **only** the `.dat` files you wish to upload

In [ ]:
# series_id to upload
ser_id = 28

# Folder containing .dat files
dat_fold = r'../../../../raw_data/2012_2016'

In [ ]:
# Process NILU data and add to db
df = cl.upload_nilu_0_1deg_dep_data(dat_fold, eng, ser_id)
df.head()

## 2. Calculate critical loads for vegetation

Critical loads for vegatation are calculated as follows:

 1. Extract raster data representing critical loads for vegetation in GeoTiff format and save locally
 
 2. Within the database, link deposition data for the time period of interest to the 0.1 degree vector grid provided by NILU
 
 3. Convert the vector grid to a raster with the same properties (extent, cell size etc.) as the critical loads grid
 
 4. Calculate the exceedance for each pixel, $i$, as $Ex_i = Dep_i - CL_i$
 
 5. Save the exceedance grid as a GeoTiff. Also calculate the total exceeded area for Norway as a whole, and the proportion exceeded in each 0.1 degree vector cell
 
 6. Write the results (i.e. the raw raster and the table & vector summaries) back to the database

In [6]:
# series_id to process
ser_id = 28

In [7]:
# Get dep values
sql = ("CREATE TABLE deposition.test_ndep AS ( "
       "SELECT ST_Transform(b.geom, 32633) AS geom, "
       "  a.cell_id, "
       "  a.n_dep "
       "FROM (SELECT cell_id, SUM(value) as n_dep "
       "      FROM deposition.dep_values_0_1deg_grid "
       "      WHERE param_id IN (1, 2) "
       "      AND series_id = %s " 
       "      GROUP BY cell_id) AS a, "
       "deposition.dep_grid_0_1deg AS b "
       "WHERE a.cell_id = b.cell_id)" % ser_id)
eng.execute(sql)

# Use 'cell_id' col as primary key
sql = ("ALTER TABLE deposition.test_ndep "
       "ADD CONSTRAINT test_ndep_pk "
       "PRIMARY KEY (cell_id)")
eng.execute(sql)

# Enforce geom type
sql = ("ALTER TABLE deposition.test_ndep "
       "ALTER COLUMN geom TYPE geometry(MULTIPOLYGON, 32633) "
       "USING ST_Multi(ST_SetSRID(geom, 32633))")
eng.execute(sql)

# Create sp. index
sql = ("CREATE INDEX test_ndep_spidx "
       "ON deposition.test_ndep "
       "USING GIST (geom)")
eng.execute(sql)

In [8]:
# Reclassify veg
sql = ("CREATE TABLE vegetation.sat_veg_120m_all_rc AS "
       "SELECT rid, "
       "  ST_Reclass(rast, 1, "
       "    (SELECT string_agg(concat('[',key,'-',key,']:', val), ',') "
       "     FROM ( "
       "       SELECT norut_code AS key, cl_100smgn_m2_yr AS val "
       "       FROM vegetation.land_class_crit_lds) lookup), "
       "    '8BUI', 255) AS rast "
       "FROM vegetation.sat_veg_120m_all")
eng.execute(sql)

# Indexes etc.
sql_list = ["ALTER TABLE vegetation.sat_veg_120m_all_rc ADD CONSTRAINT sat_veg_120m_all_rc_pk PRIMARY KEY (rid)",
            "CREATE INDEX sat_veg_120m_all_rc_idx ON vegetation.sat_veg_120m_all_rc USING gist(ST_ConvexHull(rast))"]
for sql in sql_list:
    eng.execute(sql)

In [ ]:
arr = postgis_raster_to_array(eng, 'sat_veg_crlds_div100', schema='vegetation')

In [19]:
import numpy as np
fig = plt.figure()
arr = arr[0]*1.0
arr[arr==255] = np.nan
plt.imshow(arr)

FigureCanvasNbAgg()

In [ ]:
sql = ("CREATE TABLE vegetation.exceedance AS "
       "SELECT ST_Tile(ST_MapAlgebra(vec.rast, ras.rast, '[rast1] - (100*[rast2])', NULL, 'SECOND'), 100, 100) rast "
       "FROM ( "
       "  SELECT ST_Union(ST_AsRaster(geom, (SELECT rast FROM vegetation.sat_veg_120m_all_rc LIMIT 1), '32BF', n_dep, -9999)) rast "
       "  FROM deposition.test_ndep) vec, "
       "  (SELECT ST_Union(rast) rast from vegetation.sat_veg_120m_all_rc) ras")
eng.execute(sql)


#"CREATE INDEX fields_rast_convexhull_idx ON fields_rast USING gist( ST_ConvexHull(rast) ); "
#" "
#"SELECT AddRasterConstraints('fields_rast', 'rast'); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 2); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 4); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 8); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 16); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 32); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 64); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 128); "
#"SELECT ST_CreateOverview('fields_rast'::regclass, 'rast', 256); "